In [1]:
import os
import openai
from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import random
import pandas as pd
import re
from langchain_core.output_parsers import PydanticOutputParser
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

True

In [2]:
llm = ChatOpenAI(model="o3-mini")
# llm = ChatDeepSeek(
#     model="deepseek-reasoner",
#     temperature=0.0,
# )

```RAG Tool Functions -> Function Caller -> Metrics Recommendor -> Metrics Calculator -> Analyzer -> Executor```

In [3]:
master_prompt = """

You are a DeFi Agent Launcher for Xade AI: A meta-agent that creates agents for users that can fetch, analyze DeFi data as well as execute actions based on them for the data. 
Your output is always plaintext, without any markdown.

---

You will receive the following input:

Agent Name & Description:
// The name of the agent and a short description of what it does

Query:
// A very simple, elementary query about the user's requirements which you will have to expand on and detail
 
Actions:
// Any actions the agent may be required to perform
# Eg.
#   Sentient Posting:
#   - Posting on X and XADE Terminal 
#   - Every 'x' minutes
#   - Reply on tweets by Elon Musk (@ElonMusk)
#   - Reply to mentions and quotes of oneself (@AgentName)

Specific Data Sources to Use:
// Any specific data sources to use (Separate from your recommendations)

Style:
// The style of the response the user would like to receive

Examples:
// Example interactions

---

INSTRUCTIONS:

Your task is to review the challenge, instruct and allocate tasks to your underlying LLM agents to obtain the relevant data, synthesize it, analyze it, customize responses and execute some actions.

---

You will have access to agents to automate the task. The workflow for your agents is as follows:

Data Functions Retriever → DeFi Data Fetcher & Metrics Recommender → Data Processor & Metrics Calculator → DeFi Analyzer → DeFi Executor


- Data Functions Retriever: A RAG model that can retrieve functions to fetch data relevant to the task from various sources.
  Currently, it has access to three data providers:
    - Mobula: A provider for market-based data in DeFi
    - LunarCrush: A provider for social/trending metrics in DeFi
    - CryptoPanic: A provider for news and social media posts in DeFi
You must craft the input for the RAG model keeping in mind that it can only retrieve functions to get the data, and cannot directly fetch the data or apply any filters.
If you need to identify any specific tokens on a specific blockchain or from a specific sector of DeFi (for e.g. ReFi), ask the RAG model for the functions to get those tokens.
Directly mention the kind of data you need (sentiment-related, volatility-related) or any category-specific data.
Any filters you want for the data from the RAG model should be mentioned in the instructions for the DeFi Fetcher, and not in the RAG model input

- DeFi Data Fetcher & Metrics Recommender: An agent that can call the functions retrieved by the Function Retriever to fetch data from various sources, and recommend metrics to calculate based off of that data.
  This agent will call the data functions retrieved by the Functions Retriever and organize the data for Processor. Additionally, it will also instruct the Processor on what metrics to calculate based on the data and how.
You must provide the Fetcher with a detailed description of the user's requirements that can assist it with it's recommendations for the Processor.
(NOTE: IT CANNOT YET ACCESS TWEET/X DATA)

- A DeFi Analyzer: An agent that can analyze DeFi data and provide insights based on the data. 
This agent generates a data-driven analysis and evaluation of the data while using suitable methodologies (STARE, CAR , SOAR, IDEA frameworks). 
It is smart enough to understand the context of the data and provide insights based on the data.
Based on your instructions and the data given by the Data Fetcher, it can model its analysis to:
  - Discover emerging trends
  - Analyze technical indicators such as RSI, MACD, and SMA to predict movements
  - Interpret social media metrics to gauge market sentiment and mindshare
Thus, ensure you give instructions such that the analyzer can provide insights relevant to the user's query.


- A DeFi Executor: An agent that can execute actions based on the data fetched and analyzed by the other two agents. 
This agent can devise trading strategies, execute trades, post on X and the XADE Terminal every few minutes, reply to various pre-defined indiviuals, join Twitter spaces, perform portoflio analysis, and other such actions.
It's execution will be performed by calling the relevant functions.
It will execute the actions you ask it to, based on the data fetched and analyzed by the other two agents.
It will have access to a memory of it's tasks, so it can remember its previous context and actions and adjust future actions accordingly.
Here is a list of it's abilities:
  - Post on X and the XADE Terminal periodically
  - Reply to posts by various pre-defined individuals
  - Join Twitter spaces and engage in discussions
  - Perform portfolio analysis for a given wallet address
  - Devise trading strategies based on the user's priorities and the analyzed data.
  - For text-based actions, it can customize responses based on the user's preferences.

---

When creating the instructions for the LLMs to execute, break your instructions into a logical, step-by-step order, using the specified format:
    - Primary actions must be clearly defined. You must define the end-goal that the LLM is trying to achieve through its actions.
    - Sub-actions must be indented and clearly defined, beginning on a new line.
    - Specify conditions using clear 'if...then...else' statements
    - Ensure that there is no ambiguity in any part of your plan. You must be specific and clear about what to use and how.
    - The instructions generated must be extremely detailed and thorough with explanations at every step. They must first outline the primary instruction for the LLM and then provide direct and thorough explanations

NO MARKDOWN IS ALLOWED.    
---

FORMAT INSTRUCTIONS:

{format_instructions}

---

{master_example}

TASK:
{input}

"""

In [4]:
master_example = '''

TASK:
Agent Name & Description:
Agent DAO
Performs detailed analysis on trending DAOs and provides insights on various aspects.

Query:
Comment on rising DAOs

Actions:
  Sentient Posting:
  - Posting on X and XADE Terminal 
  - Every 5 minutes
  - Reply to mentions and quotes of oneself (@AgentDAO)

Specific Data Sources to Use:
- Market Data
- Social Data
- News Feeds

Style:
Answer like an Analyst

Examples:
N.A.

INSTRUCTIONS:

1. Data Functions Retriever Input:
Query: 
Retrieve functions to
- Identify Top DAOs
- Extract information about those DAOs
- Extract news about those DAOs

2. DeFi Data Fetcher & Metrics Recommender Instructions:
Action: Fetch and filter DAO data from provided sources
- Call functions provided to fetch the required data
- Filter top 10 DAOs based on trading volume. (or closest available parameter)
- Collect information about these DAOs and organize them
- Fetch news about these DAOs and sort latest 20 items
- Recommend metrics to calculate based on fetched data
  User requirements: A detailed analysis of DAOs and their performance

3. Data Processor & Metrics Calculator Instructions:
Follow DeFi Data Fetcher's instructions to calculate metrics

4. DeFi Analyzer Instructions:
Action: Perform detailed analysis with strong growth signals using the STARE framework (Scan, Target, Analyze, Respond, Evaluate).  
  - Scan: Cross-reference information provided across data sources.  
  - Target: Rank DAOs using the metrics provided.
  - Analyze:  
    - Identify correlations in the provided data and metrics.
    - Flag key points about provided DAOs for Evaluation Report.
  - Respond:
    - Provide a detailed, structured report on provided DAOs 
    - Provide key, actionable insights as well as unique observations about data
  - Evaluate:  
    - Consider the rise in DAOs and create comments
    - Perform aspect-based analysis and evaluation of each DAO and the entire category

5. DeFi Executor Instructions:
- Primary Action: Post analysis on X/XADE Terminal every 5 minutes and reply to @AgentETH mentions.  
  - Posting Workflow:  
    - Content Structure:  
      - Header: "Rising DAO Alert: [DAO Name]".  
      - Body: "Up [X]% in trading volume + [Y]% social growth. Recent news: [Summary]."  
      - Footer: "Metrics: Trend Score [Z]/10 | Sentiment: [A]%".  
    - Priority System:  
      - Post High-Priority DAOs immediately.  
      - Post Medium-Priority DAOs if no High-Priority candidates exist.  
  - Reply to Mentions:  
    - If user asks, "Why is [DAO] trending?":  
      - Respond with: "Based on [metric] growth and [news headline], [DAO] is gaining traction due to [reason]."  
    - If user requests further analysis/comparison:  
      - Deploy data-processor-analyzer pipeline to compute and provide reply.

    Example Output for X Post  
    ```
    Rising DAO Alert: @MakerDAO  
    - Trading volume +47% in 24h  
    - Social mentions +62% (92% positive)  
    - Recent news: MakerDAO launches "Endgame" restructuring plan.  
    Metrics: Trend Score 9.2/10 | Sentiment: 92%  
    ```

'''


In [5]:
from typing import Optional

from pydantic import BaseModel, Field

class ElPlan(BaseModel):
    rag: str = Field(description="The task for the RAG model and the instructions for execution.")
    fetcher: str = Field(description="The task for the DeFi Data Fetcher and the insturctions for execution.")
    analyzer: str = Field(description="The task for the analyzer and the instructions for execution (detailed). This task must be possible to fulfill with data retreived by the Data Fetcher.")
    executor: str = Field(description="The task for the executor and the instructions to fulfill it. All specific instructions for each action can be included here as well. All miscellaneous instructions can be included here.")

parser = PydanticOutputParser(pydantic_object=ElPlan)

In [6]:
master_prompt = PromptTemplate(
    template=master_prompt,
    input_variables=["input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [7]:
input = """
Agent Name & Description:
Agent ETH
Performs detailed analysis on top Ethereum Ecosystem Projects and provides insights on various aspects.

Query:
Comment on top ETH ecosystem projects

Actions:
  Sentient Posting:
  - Posting on X and XADE Terminal 
  - Every 5 minutes
  - Reply to mentions and quotes of oneself (@AgentETH)

Specific Data Sources to Use:
- Market Data
- Social Data
- News Feeds

Style:
Answer like a DeGen

Examples:
N.A.

INSTRUCTIONS:
"""

response = llm.invoke(master_prompt.format(input=input, master_example=master_example))

In [8]:
print(response.content)

{
  "rag": "Retrieve functions to extract data on top Ethereum ecosystem projects from market data, social metrics, and news feeds. Request function calls to identify top projects by trading volume, social sentiment, and latest news updates related to Ethereum ecosystem projects.",
  "fetcher": "Fetch and filter Ethereum ecosystem project data from the provided sources. Specifically: call functions to retrieve market data showcasing trading volumes, social data indicating engagement metrics, and news feeds about Ethereum projects. Filter and organize the top 10 projects using trading volume as a primary parameter (or equivalent available metric). Additionally, collect metadata on each project to recommend subsequent metric calculations such as percentage change in volume, social sentiment scores, and news impact summaries.",
  "analyzer": "Perform a detailed, degen-style analysis using the STARE framework on the fetched Ethereum ecosystem projects data. Steps: Scan the collected market

In [9]:
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
import json

def display_agent_output(response_content):
    console = Console()
    response_content = response_content.strip('```json\n').strip('\n```')
    try:
        data = json.loads(response_content)
    except:
        console.print("[bold red]Error parsing response as JSON[/bold red]")
        return
    
    table = Table(show_header=True, header_style="bold")
    table.add_column("Component", style="italic white")
    table.add_column("Instructions", style="")
    
    for key, value in data.items():
        formatted_key = key.replace('_', ' ').title()
        table.add_row(formatted_key, value)
    
    console.print(Panel(
        table,
        title="[bold yellow]🤖 DeFi Agent Instructions[/bold yellow]",
        border_style="yellow"
    ))

In [10]:
display_agent_output(response.content)

╭────────────────────────────────────────── 🤖 DeFi Agent Instructions ───────────────────────────────────────────╮
│ ┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Component ┃ Instructions                                                                                    ┃ │
│ ┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ Rag       │ Retrieve functions to extract data on top Ethereum ecosystem projects from market data, social  │ │
│ │           │ metrics, and news feeds. Request function calls to identify top projects by trading volume,     │ │
│ │           │ social sentiment, and latest news updates related to Ethereum ecosystem projects.               │ │
│ │ Fetcher   │ Fetch and filter Ethereum ecosystem project data from the provided sources. Specifically: call  │ │
│ │           │ functions to retrieve market data showcasing trading volumes, social data indicating engagement │ │
│ │           │ metrics, and news feeds about Ethereum projects. Filter and organize the top 10 projects using  │ │
│ │           │ trading volume as a primary parameter (or equivalent available metric). Additionally, collect   │ │
│ │           │ metadata on each project to recommend subsequent metric calculations such as percentage change  │ │
│ │           │ in volume, social sentiment scores, and news impact summaries.                                  │ │
│ │ Analyzer  │ Perform a detailed, degen-style analysis using the STARE framework on the fetched Ethereum      │ │
│ │           │ ecosystem projects data. Steps: Scan the collected market, social, and news data to detect      │ │
│ │           │ emerging trends; Target and rank the projects based on calculated metrics such as trading       │ │
│ │           │ volume change, social media hype, and recent news impact; Analyze the correlations among these  │ │
│ │           │ metrics and flag standout performance indicators; Respond with a thorough, energetic and        │ │
│ │           │ concise report highlighting key insights such as rapid surges or volatility spikes; Evaluate    │ │
│ │           │ and comment on the potential of each project, including degen style observations like 'this     │ │
│ │           │ project is about to blow up' or 'massive pump incoming from news hype'.                         │ │
│ │ Executor  │ Execute actions by posting content on X and the XADE Terminal every 5 minutes and replying to   │ │
│ │           │ mentions of @AgentETH. Follow this content structure: Header: 'Top ETH Alert: [Project Name]',  │ │
│ │           │ Body: 'Rapid surge! [X]% trading volume jump with [Y]% social buzz. Latest news: [Summary].',   │ │
│ │           │ Footer: 'Trend Score: [Z]/10 | Sentiment: [A]%'. For replies to queries such as 'Why is         │ │
│ │           │ [Project] trending?', respond in degen style with context: 'Dude, [Project] is exploding thanks │ │
│ │           │ to a massive  boost and killer news:  — it's the next level on ETH!' Prioritize posting         │ │
│ │           │ high-priority projects immediately based on significant metric changes; if none exist, default  │ │
│ │           │ to medium-priority candidates. Maintain the degen tone in all postings and interactions.        │ │
│ └───────────┴─────────────────────────────────────────────────────────────────────────────────────────────────┘ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [13]:
import inspect
from typing import List, Dict, Callable
import numpy as np
import faiss
import tensorflow_hub as hub
import tensorflow as tf
import json
from mobula import *
from social import *

def process_rag_query(
    rag_instructions: str,
    functions: List[Callable],
    similarity_threshold: float = 0.7,
    verbose: bool = True
) -> Dict[str, str]:
    """
    Process RAG instructions to retrieve relevant functions.

    The search is performed on each function's docstring, while the returned result
    includes the full function source code (docstring + code). A focused query is generated
    using GPT-4o-mini from the provided instructions.

    Args:
        rag_instructions (str): The instructions to form a query for retrieval.
        functions (List[Callable]): List of function objects to consider.
        similarity_threshold (float): Cosine similarity threshold to consider a function relevant.
        verbose (bool): If True, prints detailed steps for debugging.

    Returns:
        Dict[str, str]: Mapping from function name to its full source code (as a plain string).
    """
    # 1. Build a mapping from function name to (source_code, docstring)
    if verbose:
        print("Step 1: Loading function data...")
    functions_data = {}
    for func in functions:
        name = func.__name__
        try:
            source_code = inspect.getsource(func)
        except Exception as e:
            raise ValueError(f"Could not retrieve source code for function {name}: {str(e)}")
        docstring = inspect.getdoc(func) or ""
        functions_data[name] = (source_code, docstring)
        if verbose:
            print(f"Loaded function '{name}' with docstring length: {len(docstring)} characters.")
    if verbose:
        print(f"Total functions loaded: {len(functions_data)}\n")
    
    # 2. Generate a focused query from the instructions using GPT-4o-mini.
    if verbose:
        print("Step 2: Generating focused query using GPT-4o-mini...")
    llm = ChatOpenAI(model="gpt-4o-mini")
    messages = [
        {
            "role": "system",
            "content": ("You are a DeFi Data Specialist that receives the pointers on the required information to be retrieved, and crafts an effective RAG query to find relevant data functions. "
            "For each instruction that you receive, I want you to analyze the keywords in the function and present it in a clear way that will be most effective in querying the knowledge base."
            "For each aspect that you are looking to query from the knowledge base, create a very succint and concise description of not more than 6-7 words."
            "OUTPUT FORMAT: Retrieve functions that (1).... (2).... (3)....")
        },
        {
            "role": "user",
            "content": f"INSTRUCTIONS: {rag_instructions}"
        }
    ]
    completion = llm.invoke(messages)
    rag_query = completion.content.strip()
    if verbose:
        print(f"Generated query: {rag_query}\n")
    if not rag_query:
        raise ValueError("Generated query is empty.")

    # 3. Load the Universal Sentence Encoder (USE) from TensorFlow Hub.
    if verbose:
        print("Step 3: Loading the Universal Sentence Encoder model...")
    use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
    if verbose:
        print("USE model loaded successfully.\n")

    # 4. Compute and normalize the embedding for the query.
    if verbose:
        print("Step 4: Computing embedding for the query...")
    query_embedding = use_model([rag_query]).numpy()  # shape: (1, d)
    query_embedding_norm = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)
    if verbose:
        print(f"Query embedding shape: {query_embedding_norm.shape}\n")
    
    if verbose:
        print("Step 5: Computing embeddings for function docstrings...")
    docstrings = []
    func_names = []
    for name, (source, doc) in functions_data.items():
        docstrings.append(doc)
        func_names.append(name)
    doc_embeddings = use_model(docstrings).numpy()
    doc_embeddings_norm = doc_embeddings / np.linalg.norm(doc_embeddings, axis=1, keepdims=True)
    if verbose:
        print(f"Computed embeddings for {len(docstrings)} docstrings, each of dimension {doc_embeddings_norm.shape[1]}.\n")
    
    if verbose:
        print("Step 6: Building FAISS index...")
    d = doc_embeddings_norm.shape[1]
    index = faiss.IndexFlatIP(d)
    index.add(doc_embeddings_norm.astype('float32'))
    if verbose:
        print("FAISS index built and embeddings added.\n")
    
    # 7. Search the index with the query embedding over all functions.
    if verbose:
        print("Step 7: Searching the index with the query embedding...")
    k = len(func_names)
    similarities, indices = index.search(query_embedding_norm.astype('float32'), k)
    similarities = similarities[0]  # shape: (k,)
    indices = indices[0]
    if verbose:
        print("Similarity scores and indices:")
        for sim, idx in zip(similarities, indices):
            print(f"Index: {idx}, Function: {func_names[idx]}, Similarity: {sim:.4f}")
        print("")
    
    # 8. Select function indices that exceed the similarity threshold.
    if verbose:
        print("Step 8: Selecting functions based on similarity threshold...")
    selected_indices = [idx for idx, sim in zip(indices, similarities) if sim >= similarity_threshold]
    if not selected_indices:
        if verbose:
            print("No functions met the threshold; defaulting to top 10 similar functions.")
        selected_indices = indices[:10].tolist()
    if verbose:
        print("Selected function indices:", selected_indices, "\n")
    
    # 9. Build the results: mapping function name to its full source code.
    if verbose:
        print("Step 9: Building results with full source code...")
    results = {}
    for idx in selected_indices:
        func_name = func_names[idx]
        source_code = functions_data[func_name][0]
        results[func_name] = source_code
        if verbose:
            print(f"Added function '{func_name}' to results.")
    if verbose:
        print(f"\nTotal functions retrieved: {len(results)}\n")
    
    return results
# --- Example Usage ---

# Assume these function objects are already loaded in the notebook.
functions = [
Mobula.get_all_assets,
Mobula.get_blockchains,
Mobula.get_market_blockchain_pairs,
Mobula.get_market_blockchain_stats,
Mobula.get_cefi_funding_rate,
Mobula.get_feed_create,
Mobula.get_market_history_pair,
Mobula.get_market_history,
Mobula.get_market_multi_data,
Mobula.get_market_multi_history,
Mobula.get_market_nft,
Mobula.get_market_pair,
Mobula.get_market_pairs,
Mobula.get_market_query_token,
Mobula.get_wallet_nfts,
Mobula.get_wallet_transactions,
Mobula.get_wallet_history,
Mobula.get_wallet_multi_portfolio,
Mobula.get_metadata,
Mobula.get_multi_metadata,
Mobula.get_metadata_categories,
Mobula.get_metadata_news,
Mobula.get_metadata_trendings,
Mobula.get_market_query,
Mobula.get_market_sparkline,
Mobula.get_market_token_holders,
Mobula.get_market_token_vs_market,
Mobula.get_market_total,
Mobula.search,
Mobula.get_market_data,
Mobula.get_wallet_portfolio,
Mobula.get_blockchain_pairs,
LunarCrush.get_coin_data, 
LunarCrush.get_coin_metadata, 
LunarCrush.get_nft_data, 
LunarCrush.get_coins_list, 
LunarCrush.get_topic_summary, 
CryptoPanic.get_news_and_posts
]

rag_instructions = json.loads(response.content)["rag"]

results = process_rag_query(rag_instructions, functions, similarity_threshold=0.7, verbose=True)

# Print the names and full source code of the retrieved functions.
for name, code in results.items():
  print(f"--- {name} ---")
  print(code)
  print("\n")

Step 1: Loading function data...
Loaded function 'get_all_assets' with docstring length: 1147 characters.
Loaded function 'get_blockchains' with docstring length: 1716 characters.
Loaded function 'get_market_blockchain_pairs' with docstring length: 1088 characters.
Loaded function 'get_market_blockchain_stats' with docstring length: 600 characters.
Loaded function 'get_cefi_funding_rate' with docstring length: 658 characters.
Loaded function 'get_feed_create' with docstring length: 196 characters.
Loaded function 'get_market_history_pair' with docstring length: 879 characters.
Loaded function 'get_market_history' with docstring length: 738 characters.
Loaded function 'get_market_multi_data' with docstring length: 1405 characters.
Loaded function 'get_market_multi_history' with docstring length: 334 characters.
Loaded function 'get_market_nft' with docstring length: 357 characters.
Loaded function 'get_market_pair' with docstring length: 839 characters.
Loaded function 'get_market_pairs